## Molecular dynamics simulation for Argon atoms

Here we want to simulate the behaviour of a system of argon atoms. Therefore we assume pair interactions take place with a Lennard Jones potential. The simulation is computed by the velocity Verlet algorithm. 

In [3]:
# Import necessary libaries
import numpy as np
import pylab
from mpl_toolkits.mplot3d import Axes3D
import anim_md2
from numba import jit
import cProfile
import matplotlib.pyplot as plt
import math
%matplotlib inline


In [4]:
# Settings for figures
figureparams = {'axes.labelsize': 20,
           'axes.titlesize': 20,
           'axes.linewidth': 1.3,
           'font.size': 16,
           'legend.fontsize': 15,
           'font.family': 'serif',
           'font.serif': 'Computer Modern Roman',
           'xtick.labelsize': 15,
           'xtick.major.size': 5.5,
           'xtick.major.width': 1.3,
           'ytick.labelsize': 15,
           'ytick.major.size': 5.5,
           'ytick.major.width': 1.3,
           'text.usetex': True,
           'figure.autolayout': True}
plt.rcParams.update(figureparams)

### Initialize constants in natural units

In [5]:
#Physical constants in natural units
kb=1                # Boltzman constant m^2 kg s^-2 K^-1
temp = 1            # Temperature in Kelvin/120 
eps = 1             # Depth of potential well [J]
sig = 1             # Width of the potential well 
mass = 1            # mass of argon atom 
rmin = sig*2**(2/3) # Distance for minimum V

# time parameters
dt = 4e-3           # time steps
nt = 2500           # number of timesteps for the simulation


# Cut off distances
rc2 = (sig*3)**2    # cut off length squared
rc = np.sqrt(rc2)   # Cut off length

### Calculate system size
For a given density $\rho$ and number of unit cells in each direction $m$

In [6]:
def systemsize(m, density):
    a = rmin*(density)**(-1/3)
    box_len = a*m     # System Size
    n = 4*m**3        # Number of atoms if you have m unit cells in each direction
    return box_len,n,a

# Initialization 
### Position 

For position we start with an FCC latice. Each unit cell consists of 4 atoms on the positions (0,0,0), (a/2,a/2,0), (a/2,0,a/2) and (0,a/2,a/2). We stack m unit cells in x,y,z directions. The velocity vector is a N x 3 vector. 

### Velocity
For the velocity we have again a N x 3 vector where the three columns are the velocity components in x,y,z direction. The velocity components are drawn from the maxwell boltzman distribution.
After having taking the velocities from the maxwell boltzman distribution we subtract the average velocity in each direction to get a total momentum of zero. We rescale the velocity to satisfy the equipartition theorem.
$ E=\frac{3}{2}Nk_bT=\frac{1}{2}mv^2 $.


In [7]:
def initialize(n, a, T, m):
    # initialize positions of atoms
    x = np.zeros([3,n])
    aa = 0
    for i in range(m):
        for j in range(m):
            for k in range(m):
                x[:,aa] = [i*a, j*a, k*a]
                x[:,aa+1] = [i*a + a/2, j*a + a/2, k*a]
                x[:,aa+2] = [i*a + a/2, j*a, k*a + a/2]
                x[:,aa+3] = [i*a, j*a + a/2, k*a + a/2]
                aa +=4
    
    # initialize velocity of atoms.
   
    # take 3N velocity components from gaussian distribution 
    v = np.random.normal(0, (T)**(1/2), (3,n))
    
    # Substract average velocity to have an total momentum of zero.
    v = v - np.transpose(np.sum(v, axis=1)/n *np.ones((n,1)))
    
    # rescale the velocity to satisfy equipartition theorem
    v = v*(np.sum(v*v)/(3*(n - 1)*kb*T))**(-1/2)
    
    return(x, v)


### Force calculation
To determine the force on each particle we use the Lennard Jones potential. The calc_force function loops over all particle pairs taking the nearest copy of all the paticles. If the distance between two particles is smaller than rc the cut off distance, the force and potential are determined.

The  numba command @jit is used to speed up the calculation. 

In [8]:
@jit
def calc_force(x, n, box_len):
    Fx = np.zeros(n)
    Fy = np.zeros(n)
    Fz = np.zeros(n)
    VLJ = 0
    Virial=0
    for i in range(n):
        for j in range(i):
            # Calculate the distance between particle i and j. take the nearest copy from the minimal image convention
            dx = x[0, j] - x[0, i]
            dx -= np.rint(dx/box_len)*box_len
            dy = x[1, j] - x[1, i]
            dy -= np.rint(dy/box_len)*box_len
            dz = x[2, j] - x[2, i]
            dz -= np.rint(dz/box_len)*box_len
            dr2 = dx*dx + dy*dy + dz*dz
            
            if  dr2 < rc2:
                dr_2 = 1/dr2
                dr_6 = dr_2*dr_2*dr_2
                FLJ_dr_ji = 4*dr_6*dr_2*(12*dr_6 - 6) #Force between particle j and i
                VLJ += 4*dr_6*(dr_6 - 1)
                
                Fx[i] -= FLJ_dr_ji*dx
                Fx[j] += FLJ_dr_ji*dx
                Fy[i] -= FLJ_dr_ji*dy
                Fy[j] += FLJ_dr_ji*dy
                Fz[i] -= FLJ_dr_ji*dz
                Fz[j] += FLJ_dr_ji*dz
                
                # Virial expansion calculation for cut off regime
                Virial+= dr2*FLJ_dr_ji
                
            
    F=np.array([Fx,Fy,Fz])
    return F,VLJ, Virial

## Simulation function

Here we will loop over timesteps and calculate the new positions (r_temp), the new velocity(v_temp) and the force at time t using the Verlet algorithm 
$ \boldsymbol{r}(t+dt)=\boldsymbol{r}(t)+dt*\boldsymbol{v}(t)+1/2*dt^2*\boldsymbol{F}(t)$

$\boldsymbol{v}(t+dt)=\boldsymbol{v}(t)+dt*\frac{\boldsymbol{F}(t+dt)+\boldsymbol{F}(t)}{2}$

We can implement a thermostate into the system. This means that we want to keep the temperature constant and therefore the kinetic energy constant ($E_{kin}=\frac{3}{2}Nk_bT$)

The simulation functon returns the final positions and velocities, the kinetic and potential energies and the virial pressure component for $r<r_c$

In [9]:
def simulate(n, box_len, temp, x, v, nt, dt, momcheck=False, Thermostate=False):

    VLJ = np.zeros(nt)
    Ekin = np.zeros(nt)
    
    r_temp = x
    v_temp = v
    
    Virial1 = np.zeros(nt)
    Fo = calc_force(x, n, box_len)[0]
 
    for it in range(nt):
        
        #New position of the particles following from verlet
        r_temp = r_temp + v_temp*dt + Fo*0.5*dt*dt
        r_temp = r_temp%box_len #keep particle in the box
        
        # Calculate new Force
        Fn,VLJ[it],Virial1[it] = calc_force(r_temp,n,box_len)
        
        # Calculate new velocity using F_temp and F_temp_2
        v_temp = v_temp + 0.5*(Fn + Fo)*dt
        Fo = Fn
        
        # Determine the kinetic Energy
        Ekin[it] = 0.5*np.sum(v_temp*v_temp)
        
        # Now we make a thermostate in to the system. Therefore we are going to rescale the velocity
        if (Thermostate):
            v_temp = v_temp*((3*n*kb*temp/mass)/np.sum(v_temp*v_temp))**(1/2)  
           
    # check whether the momentum is conserved
    if momcheck:
        print('Momentum in each direction at the start of simulation:', np.sum(v,axis=1))
        print('Momentum in each direction at the end of simulation:',np.sum(v_temp,axis=1))
        print('Momentum_initial/Momentum_end=',np.sum(v,axis=1)/np.sum(v_temp,axis=1))
        
    return r_temp, v_temp, Ekin, VLJ, Virial1
    

### Correlation Function

To determine the correlation function we first determine the distances between the particle pairs after the last time step. Here the system has already equilibrized. The correlation function is found in terms of np(r):

$$g(r)=\frac{2V}{N(N-1)}\Big[\frac{<np(r)>}{4\pi r^2 \Delta r}\Big] $$

Here np(r) is the number of particle pairs that have a distance in the range of r$\pm\Delta r/2 $

In [10]:
def Correlation_function(xe):
    distances = np.ones(n//2*n + (n//2 + 1)*n%2)
    ii = 0
    for i in range(n):
            for j in range(i):
                dx = xe[0,j] - xe[0,i]
                dx -= np.rint(dx/box_len)*box_len
                dy = xe[1,j] - xe[1,i]
                dy -= np.rint(dy/box_len)*box_len
                dz = xe[2,j] - xe[2,i]
                dz -= np.rint(dz/box_len)*box_len
                dist = (dx*dx + dy*dy + dz*dz)**(1/2)
                distances[ii] = dist
                ii = ii+1
               
    #Histogram of distances
    histgr = np.histogram(distances,bins=50,range=[0, box_len*.5])
    histgr_y = histgr[0] # Values of the histogram 
    histgr_x = histgr[1] # bin edges of the histogram
    
    # Center of bins:
    dx = (histgr_x[1]-histgr_x[0])/2
    histgr_xc = np.linspace(dx,histgr_x[-1]-dx,len(histgr_y))

    # Histogram for correlation function:g(r)~<n(r)>/r^2
    histg_y = 2*(box_len)**3*(n*(n-1))**(-1)*histgr_y/(4*math.pi*(histgr_xc)**2*2*dx)
    return histgr_xc, histg_y

### Mean and standard Deviation
In order to determine the standard deviation of physical quantities of our simulation we can use the method 'Data-Blocking'.
In Data-Blocking the values of the physical quantity of interest are listed in a file. This file is then chopped in blocks that are larger than the correlation time but sufficiently small to have enough blocks.
For each block the average is calculated: 
For blocks of size m, the jth block average is
    
$\displaystyle \overline{a_j}=\frac{1}{m} \sum_{k=jm+1}^{m(j+1)} a_k $

We can calculate the error as the standard deviation of the uncorrelated blocks:

$\sigma^2=\overline{A^2}-\bar{A}^2$

In [11]:
def Error(datavector, nblocks):
    
    # Divide the datavector in nblocks and calculate the average value for each block
    datavector1 = datavector[0:len(datavector) - len(datavector)%nblocks]
    data_block = np.reshape(datavector1,(nblocks,-1))
    # Used to data block specific heat
    blockmean = np.mean(data_block,axis=1) 
    blockstd = np.std(data_block,axis=1)
    # Calculate <A> en <A^2>
    Mean = np.mean(blockmean)    
    # Standard deviation
    std = np.std(blockmean)
    return Mean, std, blockmean, blockstd

## Running simulation
We run the simulation 2 times for nt steps. The first time we run the simulation so that the system can reach equilibrium. The end positions and velocities of the first run are used as initial positions and velocities for the second run. During the second run physical quantaties can be det

In [ ]:
# Initialize position and velocity
density = 0.8
m = 6
nt=2500
dt = 4e-3
box_len,n,a = systemsize(m, density)
temp = 1
nocutoff = False
x, v = initialize(n, a, temp, m)
# First run the simulation to equilibrate in nt steps keeping the temperature fixed
xe, ve = simulate(n, box_len, temp, x, v, nt, dt, Thermostate = True)[0:2]
# Run the simulation for nt steps to compute the final positions, velocity, kinetic and potential energy
xf, vf, Ekin, VLJ, Virial_cutoff = simulate(n, box_len, temp, xe, ve, nt, dt, Thermostate = False)

## Calculating and plotting
Of the energy, temperature and correlation function

In [ ]:
# Energy plot
plt.figure(1)
plt.title('Energy')
plt.plot(range(nt),Ekin)
plt.plot(range(nt),VLJ)
plt.plot(range(nt),Ekin + VLJ)
plt.legend(['Kinetic Energy','Potential Energy','Total Energy'], loc = 0)

# Potential Energy
V_2 = 2*np.pi*density*(n-1)*(1/3*rc**(-3) - 1/9*rc**(-9))*np.ones(nt) # Cutoff correction for potential energy
U = VLJ + V_2
U_mean, U_error = Error(U, 5)[0:2]
print('Final potential energy per particle is U=%.3f  with error %.3f' %(U_mean/n, U_error/n))

# Temperature
T = Ekin*2/(3*n)
T_mean, T_error = Error(T, 5)[0:2]
print('Final temperature is T=%.3f  with error %.3f' %(T_mean, T_error))

#Correlation function

gr_x_hist = np.zeros(50)
gr_y_hist = np.zeros(50)

histgr_xc,histg_y = Correlation_function(xe)

# Plot correlation function
plt.figure(2)
plt.plot(histgr_xc,histg_y)
plt.title('Correlation Function for T=%d  after %d steps' %(temp,nt))
plt.xlabel('r(\sigma)')
plt.ylabel('g')
plt.savefig('.\Figures\Correlation-function_solid.pdf')

## Pressure and Virial

To calculate the pressure of a molecular dynamics simulation the virial pressure is often obtained. 

$P=\frac{k_b T N}{V}\Big[1-\frac{1}{3Nk_b T} \Big<\sum_i \frac{\partial U}{\partial r_i}\bullet r_i \Big>\Big]$

The expectation value can easily be calculated from the force from the particle pair within hte cut-off range $r<r_c$

The contribution of particles with $r>r_c$ can analytcaly be calculated and the pressure can then be computed as follow: (equation 8.19 computational physics j.m. Thijssen):

$\frac{P}{nk_b T} = 1-\frac{1}{3Nk_b T} \Big< \sum_i \sum_{j>i} r_{ij} \frac{\partial U(R)}{\partial r_{ij}} \Big>_{cut-off} -\frac{2\pi N}{3k_b TV} \int_{r_cut off}^{\infty} r^3 \frac{\partial U(r)}{\partial r} g(r) dr$

Here g(r) the correlation function can be replaced by 1 since for large r.


So the Virial Pressure:

$\frac{P}{nk_b T} = 1-\frac{1}{3Nk_b T} \Big< \sum_i \sum_{j>i} r_{ij} F_{ij} \Big>_{cut-off} -\frac{\pi N \epsilon}{6k_b TV}\Big[ \frac{32 \sigma^6}{r_{cut-off}^3}-\frac{64\sigma^{12}}{3r_{cut-off}^9}\Big]$ 


In [ ]:
# Calculate analytical tail of virial pressure
Virial_2 = np.pi*n*(6*temp*box_len**3)**(-1)*(32*rc**(-3)-64/3*rc**(-9))*np.ones(nt)

# Add all components of the pressure and determine the mean and error
Pvirial= n*kb*temp*box_len**(-3)*(1 + (3*n*kb*temp)**(-1)*Virial_cutoff - Virial_2)
P_mean, P_error = Error(Pvirial,5)[0:2]
P_mean_kbT = 1/temp*P_mean
P_error_kbT = 1/temp*P_error

print('Final Pressure is P/(density*beta)=%.3f  with error %.3f' %(P_mean_kbT/density, P_error_kbT/density))

## Heat Capacity

From lebowits (equation 7.37 from computational physics)

$\frac{<\delta K^2>}{<K>^2} =\frac{2}{3N} \big( 1-\frac{3N}{2C_v}\big)$

$ C_v=\Big[\frac{2}{3N}-\frac{<\delta K^2>}{<K>^2}\Big]^{-1}$

Here $<\delta K^2>$ is the standard deviation of the kinetic energy.


In [ ]:
# Calculate Expectation  value and variation
Exp_K, Std_K = Error(Ekin,5)[2:4]
Exp_K2 = Exp_K*Exp_K
Var_K = Std_K*Std_K
# Calculate Heat Capacity
Cv = (2/(3*n) - Var_K/(Exp_K**2))**(-1)
Cv_err = np.std(Cv)
Cv = np.mean(Cv)
print('Specific heat is Cv=%.3f  with error %.3f' %(Cv/n, Cv_err/n))

## Correlation time

To determine the correlation time we will look at teh standard deviation for the kinetic energy. For a simulation of nt steps the standard deviation is determined for different block length using Data-blocking. The correlation time is approximately the value where the standard deviation approaches a constant value for increasing blocklengths. 

In [ ]:
# Plot to determine optimal block size for data blocking 

std=np.zeros(nt-1)
for i in range(nt-1):
    datavector_eq = Ekin
    nblocks = i + 1
    # Divide the datavector in nblocks and calculate the average value for each block
    datavector_eq1 = datavector_eq[0:len(datavector_eq)-len(datavector_eq)%nblocks]
    data_block = np.reshape(datavector_eq1,(nblocks,-1))
    block = np.mean(data_block,axis=1)
    blockvar = np.std(data_block,axis=1)
    Mean = np.mean(block)
    # Standard deviation
    std[i] = np.std(block)
x=np.linspace(1,500,num=nt-1)
x2=np.rint(nt/x)
plt.figure()
#plt.title('Energy')
plt.plot(x2,std/50)
plt.xlabel(r'block length')
plt.ylabel(r'error')